# (E06)6th-project-music_writer
***

## 1. 데이터 읽어오기

     Song Lyrics를 https://www.kaggle.com/paultimothymooney/poetry/data 에서 받아 저장해줍니다.

In [1]:
import glob
import os

import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 텐서플로우

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['', '\t\t\t“There must be some way out of here,” said the joker to the thief', '“There’s too much confusion, I can’t get no relief']


## 2. 데이터 정제하기
      
      저장된 문장들을 확인해봅니다.

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인
        
    print(sentence)

			“There must be some way out of here,” said the joker to the thief
“There’s too much confusion, I can’t get no relief
Businessmen, they drink my wine, plowmen dig my earth
None of them along the line know what any of it is worth”
“No reason to get excited,” the thief, he kindly spoke
“There are many here among us who feel that life is but a joke
But you and I, we’ve been through that, and this is not our fate
So let us not talk falsely now, the hour is getting late”


      정규표현식을 이용해 문장을 다듬어 줍니다. 확인 결과 제대로 작동합니다.

In [3]:
def preprocess_sentence(sentence):
    s = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    s = re.sub(r"([?.!,¿])", r" \1 ", s)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    s = re.sub(r'[" "]+', " ", s)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    s = re.sub(r"[^a-zA-Z?.!,¿]+", " ", s)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    s = s.strip()

    s = '<start> ' + s + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return s

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


      정제된 문장들을 리스트에 넣어줍니다. 이때 끝이 ':'로 끝나거나 문장이 없는 경우 건너뜁니다.

In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue

    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> there must be some way out of here , said the joker to the thief <end>',
 '<start> there s too much confusion , i can t get no relief <end>',
 '<start> businessmen , they drink my wine , plowmen dig my earth <end>',
 '<start> none of them along the line know what any of it is worth <end>',
 '<start> no reason to get excited , the thief , he kindly spoke <end>',
 '<start> there are many here among us who feel that life is but a joke <end>',
 '<start> but you and i , we ve been through that , and this is not our fate <end>',
 '<start> so let us not talk falsely now , the hour is getting late <end>',
 '<start> all along the watchtower , princes kept the view <end>',
 '<start> while all the women came and went , barefoot servants , too <end>']

In [5]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.
    for length in tensor:
        if len(length) > 15:  #문장의 토큰 길이가 15이상일 경우
            length.pop()      #pop으로 제거해줍니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen = 15, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  65  280   27 ...   10    6 2474]
 [   2   65   16 ... 5339    3    0]
 [   2    1    4 ...    3    0    0]
 ...
 [   2   75   46 ...    3    0    0]
 [   2   49    4 ...    0    0    0]
 [   2   13  636 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fd1a9434850>


In [6]:
print(tensor[:5])

[[  65  280   27   99   85   56   19   93    4  107    6 6481   10    6
  2474]
 [   2   65   16  103  179 2963    4    5   32   15   43   38 5339    3
     0]
 [   2    1    4   45  470   13  948    4    1  993   13  671    3    0
     0]
 [   2  889   19  110  601    6  431   34   40  397   19   11   26  743
     3]
 [   2   38  568   10   43 2834    4    6 2474    4   55 5041 1763    3
     0]]


      토큰화 된 단어 리스트를 확인합니다.

In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  65  280   27   99   85   56   19   93    4  107    6 6481   10    6]
[ 280   27   99   85   56   19   93    4  107    6 6481   10    6 2474]


## 3.  평가 데이터셋 분리

      데이터들을 훈련셋과 검증셋으로 나눠줍니다.

In [9]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, shuffle=True)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


In [10]:
import tensorflow as tf

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

# for train
train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)
print(train_dataset)

# for test
test_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)
print(test_dataset)

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>
<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>


## 4. 훈련 모델 설계
    
      모델을 만들고 학습한 후, val_loss와 생성된 문장을 확인합니다.
      모델은 RNN을 두 층으로 만들어 확인해봅시다.

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256 # 단어 벡터의 차원수입니다.
hidden_size = 1024 # 생각하는 과정 층의 수인데 요약하자면 뱃사공의 수입니다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

      모델의 input 모양을 자동으로 결정해주는 코드입니다.
      tensor 의 형태가 256, 14, 12001로 각각 batch size와 길이, 단어 수를 뜻합니다.

In [12]:
for enc_train, dec_train in train_dataset.take(1): break
model(enc_train)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 4.89671984e-05,  1.91120009e-04,  2.52723199e-04, ...,
         -5.89945421e-06, -1.57434246e-04, -2.01262053e-04],
        [ 5.97140133e-05,  3.17622675e-04,  4.22464305e-04, ...,
         -1.71261636e-05, -1.39817537e-04, -4.41581215e-04],
        [ 6.01241736e-05,  4.40415752e-04,  5.00677619e-04, ...,
          2.89606483e-04, -2.64556991e-04, -4.08167427e-04],
        ...,
        [ 2.03412492e-04, -6.12976582e-05,  2.41857233e-05, ...,
         -1.50883745e-04, -1.74772157e-03, -8.72927485e-04],
        [ 2.55839579e-04, -1.75313355e-04, -1.63948251e-04, ...,
         -3.50443850e-04, -1.66532316e-03, -8.10133468e-04],
        [ 3.61742510e-04, -5.95992242e-05, -1.28375905e-04, ...,
         -3.78162717e-04, -1.30274124e-03, -1.05609687e-03]],

       [[ 4.89671984e-05,  1.91120009e-04,  2.52723199e-04, ...,
         -5.89945421e-06, -1.57434246e-04, -2.01262053e-04],
        [ 2.80378561e-04,  2.84652662e-04,  5

      생성된 모델을 확인하고 학습을 시작하고 결과를 관찰해봅니다.

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
history_model = model.fit(train_dataset, epochs=10, validation_data=test_dataset, verbose=1)

Epoch 1/10
549/549 [==============================] - 78s 143ms/step - loss: 3.7133 - val_loss: 3.3710
Epoch 2/10
549/549 [==============================] - 79s 145ms/step - loss: 3.2346 - val_loss: 3.1779
Epoch 3/10
549/549 [==============================] - 81s 147ms/step - loss: 3.0532 - val_loss: 3.0497
Epoch 4/10
549/549 [==============================] - 86s 156ms/step - loss: 2.9101 - val_loss: 2.9538
Epoch 5/10
549/549 [==============================] - 85s 155ms/step - loss: 2.7881 - val_loss: 2.8801
Epoch 6/10
549/549 [==============================] - 83s 151ms/step - loss: 2.6777 - val_loss: 2.8171
Epoch 7/10
549/549 [==============================] - 83s 151ms/step - loss: 2.5738 - val_loss: 2.7672
Epoch 8/10
549/549 [==============================] - 80s 146ms/step - loss: 2.4742 - val_loss: 2.7249
Epoch 9/10
549/549 [==============================] - 80s 146ms/step - loss: 2.3803 - val_loss: 2.6872
Epoch 10/10
549/549 [==============================] - 80s 146ms/step - l

      Epoch가 10회일 때 val loss가 약 2.6으로 목표치인 2.2보다 높습니다.
      계속해서 텍스트 생성 함수를 작성하고 생성된 문장을 확인해 봅니다.

In [15]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [16]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m gonna be your fantasy <end> '

      그럴싸한 문장이 나온 것을 확인할 수 있습니다. 계속해서 성능 향상을 시도해봅니다.

## 5. 향상된 학습 모델 설계하기

      드롭아웃 추가하기, 배치 정규화, 레이어 수 늘리기 등 여러가지 모델을 설계해보고 학습해 본 결과 아래의 모델의 성능이 가장 뛰어났습니다.

In [17]:
del corpus
del tensor # 메모리 확보

In [18]:
model.reset_states()# 메모리 확보

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        
        self.rnn1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True) 
        self.rnn_d1 = tf.keras.layers.Dropout(0.5)  # 해당 레이어는 드롭아웃입니다
        #self.rnn_bn1 = tf.keras.layers.BatchNormalization() #해당 레이어는 배치 정규화입니다
        
        #self.rnn2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        #self.rnn_d2 = tf.keras.layers.Dropout(0.5)
        #self.rnn_bn2 = tf.keras.layers.BatchNormalization() #해당 블록은 추가된 층입니다
        
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        
        out = self.rnn1(out)
        out = self.rnn_d1(out)
        #out = self.rnn_bn1(out)
        
        #out = self.rnn2(out)
        #out = self.rnn_d2(out)
        #out = self.rnn_bn2(out)

        out = self.linear(out)
        
        return out
    
embedding_size = 256*2 # 늘릴수록 소폭의 성능 향상을 확인할 수 있었습니다
hidden_size = 1024*4 # 히든 레이어가 많으면 문제가 생길 수 있다고 했지만 확인 결과 늘일수록 학습시간은 오래걸리지만 가장 눈에띄는 성능 향상을 보였습니다
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

for enc_train, dec_train in train_dataset.take(1): break
model(enc_train)

model.summary()

optimizer = tf.keras.optimizers.Nadam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
history_model = model.fit(train_dataset, epochs=10, validation_data=test_dataset, verbose=1)

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  6144512   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  75513856  
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  49168097  
Total params: 130,826,465
Trainable params: 130,826,465
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
549/549 [==============================] - 349s 635ms/step - loss: 3.7303 - val_loss: 3.2402
Epoch 2/10
549/549 [==============================] - 332s 605ms/step - loss: 3.0773 - val_loss: 2.9548
Epoch 3/10
549/549 [======

      확인 결과 Epoch=10에서 과적응이 보였지만 2.2대에 진입함으로 지금까지 만들었던 모델 중 가장 뛰어난 성능을 보였습니다.

In [19]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you <end> '

In [20]:
generate_text(model, tokenizer, init_sentence="<start> i ", max_len=20)

'<start> i m not the only one <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> love", max_len=20)

'<start> love is a beautiful thing <end> '

In [26]:
generate_text(model, tokenizer, init_sentence="<start> one", max_len=20)

'<start> one more time fore i go <end> '

      문장이 잘 생성되는 것을 확인했습니다....

***
# 결론 및 총평

      해당 모델을 설계하고 성능을 확인하는 동안 학습시간이 너무 오래 걸려서 시간의 낭비가 심했다. 게다가 층을 늘리든, 어떠한 정규화와 드롭아웃을 추가해줘도 과적응이 일어나거나 성능의 향상이 미미해 목표치에 미달하는 경우가 많았다. 결국 해당 예제에서 추천하지 않은 히든 레이어 수를 늘리는 것이 가장 효과적이였다.
      
      느끼기에는 학습모델의 val loss값이 어떻든 생성되는 문장의 수준은 비슷비슷한 거 같아서 아쉬웠다. 심지어 문장이 통채로 어떤 노래의 가사였던 경우도 있었다. 전체적으로 들인 시간과 노력에 비해 향상은 보이지 않고 지루해서 피곤한 과정이였다.